In [1]:
## Boiler plate code common to many notebooks.  See the TestFilesCommonCode.ipynb for details
from __future__ import print_function
%run TestFilesCommonCode.ipynb

SimpleITK Version: 0.9.1
Compiled: Sep 28 2015 10:07:41



In [2]:
#dwi_fn='/scratch/TESTS/IpythonNotebook/20160615_HCPWF/mainWF/Outputs/DWI_Baseline.nrrd'
dwi_fn='/scratch/TESTS/IpythonNotebook/20160615_HCPWF/mainWF/Outputs/DWI_corrected_alignedSpace.nrrd'

t1_fn='/scratch/TESTS/IpythonNotebook/20160615_HCPWF/mainWF/HCPWorkflow_CACHE_105115/PreprocessingWorkflow_CACHE_105115/ResampleToAlignedDWIResolution/mapflow/_ResampleToAlignedDWIResolution0/StrippedT1_125.nrrd'
t2_fn='/scratch/TESTS/IpythonNotebook/20160615_HCPWF/mainWF/HCPWorkflow_CACHE_105115/PreprocessingWorkflow_CACHE_105115/ResampleToAlignedDWIResolution/mapflow/_ResampleToAlignedDWIResolution1/StrippedT2_125.nrrd'

label_fn='/scratch/TESTS/IpythonNotebook/20160615_HCPWF/mainWF/HCPWorkflow_CACHE_105115/PreprocessingWorkflow_CACHE_105115/ResampleToAlignedDWIResolution/mapflow/_ResampleToAlignedDWIResolution2/DWIBrainMask.nrrd'

OutputDir='/scratch/TESTS/IpythonNotebook/20160615_HCPWF/2_SRWF/test_tune_parameters/outImageFiles'
MatlabFilesDir='/scratch/TESTS/IpythonNotebook/20160615_HCPWF/2_SRWF/test_tune_parameters/matlabFiles'

In [3]:
import os
import glob
import sys

#\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/
#####################################################################################
#     Prepend the shell environment search paths
#PROGRAM_PATHS = '/scratch/NAMICExternalProjects/release_20150715/bin'
PROGRAM_PATHS = '/scratch/BS/release-BSR/bin'
PROGRAM_PATHS = PROGRAM_PATHS.split(':')
PROGRAM_PATHS.extend(os.environ['PATH'].split(':'))
os.environ['PATH'] = ':'.join(PROGRAM_PATHS)

CUSTOM_ENVIRONMENT=dict()

# Platform specific information
#     Prepend the python search paths
PYTHON_AUX_PATHS = '/scratch/BS/BRAINSTools/AutoWorkup'
PYTHON_AUX_PATHS = PYTHON_AUX_PATHS.split(':')
PYTHON_AUX_PATHS.extend(sys.path)
sys.path = PYTHON_AUX_PATHS

import SimpleITK as sitk
import nipype
from nipype.interfaces.base import CommandLine, CommandLineInputSpec, TraitedSpec, File, Directory
from nipype.interfaces.base import traits, isdefined, BaseInterface
from nipype.interfaces.utility import Merge, Split, Function, Rename, IdentityInterface
import nipype.interfaces.io as nio   # Data i/oS
import nipype.pipeline.engine as pe  # pypeline engine
from nipype.interfaces.freesurfer import ReconAll
from nipype.interfaces.ants import DenoiseImage
from nipype.interfaces.semtools import *

In [4]:
# 
# READ/WRITE DWI scan using pynrrd
#
import nrrd
from math import sqrt
from collections import OrderedDict

class nrrdDWIHeader:
    """A helper class for manipulating header information
    from a nrrd DWI data set into a nibabel compliant
    format"""
    def __init__(self,pynrrdDataModel):
        self.modality=None                      #Matches NRRD File
        self.global_BValue=None                 #Matches NRRD File
        self.gradientUnormalizedVectors=[None]  #Matches NRRD File

        self.gradientIndex=-1
        self.gradientBValues=[None]             #Computed
        self.gradientVectors=[None]             #Computed
        self._ConvertNrrdToNibabelDWIDataModel(pynrrdDataModel)

    def Print(self):
        print("global_BValue {0}".format(self.global_BValue) )
        print("modality {0}".format(self.modality) )
        print("gradientBValues {0}".format(self.gradientBValues) )
        print("gradientDirections {0}".format(self.gradientUnormalizedVectors) )

    def _getGradientStorageIndex(self,pynrrdDataModel):
        """The 4D index that has separate gradients.
        all other directions are the spatial dimensions."""
        gradient_index=-1;
        centerings=pynrrdDataModel['centerings']
        numdwidims=len(centerings)
        for test_index in range(0,numdwidims):
            if centerings[test_index] not in ['cell']:
                gradient_index=test_index
        self.gradientIndex = gradient_index

    def _ExtractGlobalBValue(self, pyNrrdKVUnknownMetaData):
        globalBValueString=pyNrrdKVUnknownMetaData.get(u'DWMRI_b-value','0').lstrip().rstrip()
        self.global_BValue=float( globalBValueString )

    def _ExtractUnormalizedBValues(self,pyNrrdKVUnknownMetaData,pynrrdDataModel):
        """Unnormalized data values from the nrrd files, where the
        magnitude is reflective of the scale relative to the global_BValue"""
        self._getGradientStorageIndex(pynrrdDataModel)
        numGradients = pynrrdDataModel[u'sizes'][self.gradientIndex]
        self.gradientUnormalizedVectors= np.array( [ [ None, None, None ] for x in range(0,numGradients) ] )
        gvec_Fields=pyNrrdKVUnknownMetaData.copy() #Clone so we can remove items
        for k,v in gvec_Fields.iteritems():
            if k.startswith("DWMRI_gradient_"):
                index=int(k.split('_')[2])
                self.gradientUnormalizedVectors[index,:]=np.array([ float(x) for x in v.rstrip().lstrip().split() ],
                                                    copy=True, order='C', ndmin=1)
                pyNrrdKVUnknownMetaData.pop(k)


    def _ExtractDWIModality(self,pyNrrdKVUnknownMetaData):
        self.modality=pyNrrdKVUnknownMetaData.get("modality","UNKNOWN").lstrip().rstrip()

    def _ComputeNormalizedBvecBVals(self,pyNrrdKVUnknownMetaData,pynrrdDataModel):
        self._ExtractGlobalBValue(pyNrrdKVUnknownMetaData)
        self._ExtractUnormalizedBValues(pyNrrdKVUnknownMetaData,pynrrdDataModel)
        self._ComputeNormalizedGradientsAndBValues()

    def _ComputeNormalizedGradientsAndBValues(self):
        """Following conventions of NRRD format, expand
        to normalized vectors and estimate BValues
        :return: void
        """
        numGradients=len( self.gradientUnormalizedVectors )

        self.gradientVectors = np.array(self.gradientUnormalizedVectors)
        self.gradientBValues = np.array( [ self.global_BValue ] * numGradients )
        for index in range(0,numGradients):
            gv = self.gradientUnormalizedVectors[index]
            norm=np.linalg.norm(gv)
            if norm < 1e-2:
                self.gradientVectors[index] = gv * 0.0
                self.gradientBValues[index] = 0.0
            elif ( abs( 1.0-norm ) > 1e-4 ): # Avoid rescaling if norm is almost one
                self.gradientVectors[index] = gv/norm
                b_i = self.global_BValue * (norm**2) # norm = sqrt(b_i/b_max)
                self.gradientBValues[index] = float("{0:.1f}".format(b_i))  

    def _ConvertNrrdToNibabelDWIDataModel(self, pynrrdDataModel):
        pyNrrdKVUnknownMetaData=pynrrdDataModel['keyvaluepairs']
        self._ExtractDWIModality(pyNrrdKVUnknownMetaData)
        self._ExtractGlobalBValue(pyNrrdKVUnknownMetaData)
        self._ComputeNormalizedBvecBVals(pyNrrdKVUnknownMetaData,pynrrdDataModel)

def ReadNAMICDWIFromNrrd(filename):
    nrrd_dwi_data,nrrd_dwi_header=nrrd.read(filename)
    nibabelDataModelDWI=nrrdDWIHeader(nrrd_dwi_header)
    nrrd_dwi_bvec=nibabelDataModelDWI.gradientVectors
    nrrd_dwi_bval=nibabelDataModelDWI.gradientBValues
    gradient_index=nibabelDataModelDWI.gradientIndex
    return (nrrd_dwi_data, nrrd_dwi_header, nrrd_dwi_bvec, nrrd_dwi_bval, gradient_index)

def WriteNAMICDWIToNrrd(filename, data, bvecs, bvals, options=None):
    """
    :param filename: The filename to write to disk
    :param data: The numpy 4d file to be written
    :param bvecs: The bvecs values to be written
    :param bvals: The bvals to be written
    :param options: Optional parameters to be written to the nrrd header
    :return:
    """
    keyvaluePairDict=OrderedDict()
    if options is not None:
        keyvaluePairDict=OrderedDict( options.get( 'keyvaluepairs',OrderedDict() ))

    ## First remove all existing bval/bvec fields from dictionary
    keyvaluePairDict.pop('modality',None)

    for k,v in keyvaluePairDict.iteritems():
        if k.startswith('DWMRI_'):
            keyvaluePairDict.pop(k, None)

    keyvaluePairDict[u'modality']=u'DWMRI'
    maxBvalue = max(bvals)
    keybval=u'DWMRI_b-value'
    keyvaluePairDict[keybval]=maxBvalue

    numGradients = len(bvecs)
    for index in range(0,numGradients):
        this_scale = sqrt(bvals[index] / maxBvalue)
        this_vec = [ x * this_scale for x in bvecs[index]]
        #print(bvecs[index],this_vec)

        keyvec=u'DWMRI_gradient_{:04d}'.format(index)
        # convert gradient vector value to string (only for consistency with input)
        vec_string= lambda this_vec: u''.join(nrrd._convert_to_reproducible_floatingpoint(x)+'  ' for x in this_vec).rstrip()
        keyvaluePairDict[keyvec]=vec_string(this_vec)
    options['keyvaluepairs']=keyvaluePairDict
    options['encoding']='gzip' # Always use gzip compression for DWI data
    nrrd.write(filename,data,options)

# Mask dwi_b0 image

In [5]:
def MaskDWIComponents(inputDWI,mask_fn):
    assert os.path.exists(inputDWI), "File not found: %s" % inputDWI
    assert os.path.exists(mask_fn), "File not found: %s" % mask_fn
    mask_img = sitk.ReadImage(mask_fn)
    mask_data = sitk.GetArrayFromImage(mask_img)    
    dwi_data,dwi_header,bvecs,bvals,gradient_index = ReadNAMICDWIFromNrrd(inputDWI)
    numOfComponents = dwi_data.shape[gradient_index]
    for idx in range(numOfComponents):
        dwi_3d = dwi_data[idx,:,:,:]
        #dwi_3d = np.squeeze( dwi_data.take((idx,), axis=gradient_index) )
        dwi_3d = np.transpose(dwi_3d,(2, 1, 0))
        dwi_3d = dwi_3d*mask_data
        dwi_3d = np.transpose(dwi_3d,(2, 1, 0))
        dwi_data[idx,:,:,:] = dwi_3d
    outMaskedDWIFilename = os.path.join(OutputDir, 'DWI_masked.nrrd')
    # write corrected nrrd file to disk
    WriteNAMICDWIToNrrd(outMaskedDWIFilename,dwi_data,bvecs,bvals,dwi_header)
    assert os.path.isfile(outMaskedDWIFilename), "Masked DWI file is not found: %s" % outMaskedDWIFilename
    #return outNormalizedDWIFilename

In [7]:
MaskDWIComponents(dwi_fn,label_fn)

In [ ]:
# Experiments ------

In [9]:
dwi_data,dwi_header,bvecs,bvals,gradient_index = ReadNAMICDWIFromNrrd(dwi_fn)

In [10]:
print(gradient_index)

0


In [26]:
if gradient_index != 0:
    raise ValueError('Program expects a gradient index of zero for HCP data, but gradient index is {0}'.format(gradient_index))

In [11]:
print(dwi_data.shape)
print(dwi_data.shape[gradient_index])

(108, 145, 174, 145)
108


In [12]:
# Now extract b0 component of the 4D DWI array

In [22]:
# idx=0
# dwi_b0 = np.squeeze( dwi_data.take((idx,), axis=gradient_index) )
# print(dwi_b0.shape)
# dwi_b0 = np.transpose(dwi_b0,(2, 1, 0))
# dwi_b0_img = sitk.GetImageFromArray(dwi_b0)
# myshow(dwi_b0_img)

In [23]:
# Now extract b0 component of the 4D DWI array
dwi_b0 = dwi_data[0,:,:,:]
print(dwi_b0.shape)

(145, 174, 145)


To convert the above numpy array to a b0 image, we need careful attention to the order of index and dimensions.

ITK's Image class does not have a bracket operator. It has a GetPixel which takes an ITK Index object as an argument, which is an array ordered as (x,y,z). This is the convention that SimpleITK's Image class uses for the GetPixel method as well.

While in numpy, an array is indexed in the opposite order (z,y,x).

In [ ]:
dwi_b0 = np.transpose(dwi_b0,(2, 1, 0))

In [ ]:
mask_img = sitk.ReadImage(label_fn)
myshow(mask_img)
mask_data = sitk.GetArrayFromImage(mask_img)
print(mask_data.shape)

In [ ]:
masked_dwi_b0 = dwi_b0*mask_data

In [ ]:
dwi_b0_img = sitk.GetImageFromArray(masked_dwi_b0)
myshow(dwi_b0_img)

In [ ]:
masked_dwi_b0 = np.transpose(masked_dwi_b0,(2, 1, 0))
dwi_b0_img = sitk.GetImageFromArray(masked_dwi_b0)
myshow(dwi_b0_img)